In [1]:
!sh get_data.sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11.9M    0 11.9M    0     0   649k      0 --:--:--  0:00:18 --:--:--  465k
Archive:  tmp.zip
  inflating: tulips1/PreprocessedData/MehreenSaeed/ContourParameters/four  
  inflating: tulips1/PreprocessedData/MehreenSaeed/ContourParameters/one  
  inflating: tulips1/PreprocessedData/MehreenSaeed/ContourParameters/README  
  inflating: tulips1/PreprocessedData/MehreenSaeed/ContourParameters/three  
  inflating: tulips1/PreprocessedData/MehreenSaeed/ContourParameters/two  
  inflating: tulips1/PreprocessedData/MehreenSaeed/MarkedImages/Anthony/Anthony11.00001  
  inflating: tulips1/PreprocessedData/MehreenSaeed/MarkedImages/Anthony/Anthony11.00002  
  inflating: tulips1/PreprocessedData/MehreenSaeed/MarkedImages/Anthony/Anthony11.00003  
  inflating: tulips1/PreprocessedData/MehreenSaeed/MarkedImages/Anthony/Anthony11.00004  
  in

  inflating: tulips1/PreprocessedData/MehreenSaeed/MarkedImages/Ben/Ben32.00015  
  inflating: tulips1/PreprocessedData/MehreenSaeed/MarkedImages/Ben/Ben32.00016  
  inflating: tulips1/PreprocessedData/MehreenSaeed/MarkedImages/Ben/Ben41.00001  
  inflating: tulips1/PreprocessedData/MehreenSaeed/MarkedImages/Ben/Ben41.00002  
  inflating: tulips1/PreprocessedData/MehreenSaeed/MarkedImages/Ben/Ben41.00003  
  inflating: tulips1/PreprocessedData/MehreenSaeed/MarkedImages/Ben/Ben41.00004  
  inflating: tulips1/PreprocessedData/MehreenSaeed/MarkedImages/Ben/Ben41.00005  
  inflating: tulips1/PreprocessedData/MehreenSaeed/MarkedImages/Ben/Ben41.00006  
  inflating: tulips1/PreprocessedData/MehreenSaeed/MarkedImages/Ben/Ben41.00007  
  inflating: tulips1/PreprocessedData/MehreenSaeed/MarkedImages/Ben/Ben41.00008  
  inflating: tulips1/PreprocessedData/MehreenSaeed/MarkedImages/Ben/Ben41.00009  
  inflating: tulips1/PreprocessedData/MehreenSaeed/MarkedImages/Ben/Ben41.00010  
  inflating: tul

  inflating: tulips1/PreprocessedData/MehreenSaeed/MarkedImages/Cynthea/Cynthea21.00002  
  inflating: tulips1/PreprocessedData/MehreenSaeed/MarkedImages/Cynthea/Cynthea21.00003  
  inflating: tulips1/PreprocessedData/MehreenSaeed/MarkedImages/Cynthea/Cynthea21.00004  
  inflating: tulips1/PreprocessedData/MehreenSaeed/MarkedImages/Cynthea/Cynthea21.00005  
  inflating: tulips1/PreprocessedData/MehreenSaeed/MarkedImages/Cynthea/Cynthea21.00006  
  inflating: tulips1/PreprocessedData/MehreenSaeed/MarkedImages/Cynthea/Cynthea21.00007  
  inflating: tulips1/PreprocessedData/MehreenSaeed/MarkedImages/Cynthea/Cynthea21.00008  
  inflating: tulips1/PreprocessedData/MehreenSaeed/MarkedImages/Cynthea/Cynthea21.00009  
  inflating: tulips1/PreprocessedData/MehreenSaeed/MarkedImages/Cynthea/Cynthea21.00010  
  inflating: tulips1/PreprocessedData/MehreenSaeed/MarkedImages/Cynthea/Cynthea22.00001  
  inflating: tulips1/PreprocessedData/MehreenSaeed/MarkedImages/Cynthea/Cynthea22.00002  
  inflatin

  inflating: tulips1/PreprocessedData/MehreenSaeed/MarkedImages/Don/Don31.00011  
  inflating: tulips1/PreprocessedData/MehreenSaeed/MarkedImages/Don/Don31.00012  
  inflating: tulips1/PreprocessedData/MehreenSaeed/MarkedImages/Don/Don32.00001  
  inflating: tulips1/PreprocessedData/MehreenSaeed/MarkedImages/Don/Don32.00002  
  inflating: tulips1/PreprocessedData/MehreenSaeed/MarkedImages/Don/Don32.00003  
  inflating: tulips1/PreprocessedData/MehreenSaeed/MarkedImages/Don/Don32.00004  
  inflating: tulips1/PreprocessedData/MehreenSaeed/MarkedImages/Don/Don32.00005  
  inflating: tulips1/PreprocessedData/MehreenSaeed/MarkedImages/Don/Don32.00006  
  inflating: tulips1/PreprocessedData/MehreenSaeed/MarkedImages/Don/Don32.00007  
  inflating: tulips1/PreprocessedData/MehreenSaeed/MarkedImages/Don/Don32.00008  
  inflating: tulips1/PreprocessedData/MehreenSaeed/MarkedImages/Don/Don32.00009  
  inflating: tulips1/PreprocessedData/MehreenSaeed/MarkedImages/Don/Don32.00010  
  inflating: tul

  inflating: tulips1/PreprocessedData/MehreenSaeed/MarkedImages/George/George42.00008  
  inflating: tulips1/PreprocessedData/MehreenSaeed/MarkedImages/George/George42.00009  
  inflating: tulips1/PreprocessedData/MehreenSaeed/MarkedImages/George/George42.00010  
  inflating: tulips1/PreprocessedData/MehreenSaeed/MarkedImages/George/George42.00011  
  inflating: tulips1/PreprocessedData/MehreenSaeed/MarkedImages/George/George42.00012  
  inflating: tulips1/PreprocessedData/MehreenSaeed/MarkedImages/George/George42.00013  
  inflating: tulips1/PreprocessedData/MehreenSaeed/MarkedImages/Isaac/Isaac11.00001  
  inflating: tulips1/PreprocessedData/MehreenSaeed/MarkedImages/Isaac/Isaac11.00002  
  inflating: tulips1/PreprocessedData/MehreenSaeed/MarkedImages/Isaac/Isaac11.00003  
  inflating: tulips1/PreprocessedData/MehreenSaeed/MarkedImages/Isaac/Isaac11.00004  
  inflating: tulips1/PreprocessedData/MehreenSaeed/MarkedImages/Isaac/Isaac11.00005  
  inflating: tulips1/PreprocessedData/Mehr

...

  inflating: tulips1/RawData/tulips1.A/raw/Don11e.au  
  inflating: tulips1/RawData/tulips1.A/raw/Don12e.au  
  inflating: tulips1/RawData/tulips1.A/raw/Don21e.au  
  inflating: tulips1/RawData/tulips1.A/raw/Don22e.au  
  inflating: tulips1/RawData/tulips1.A/raw/Don31e.au  
  inflating: tulips1/RawData/tulips1.A/raw/Don32e.au  
  inflating: tulips1/RawData/tulips1.A/raw/Don41e.au  
  inflating: tulips1/RawData/tulips1.A/raw/Don42e.au  
  inflating: tulips1/RawData/tulips1.A/raw/George11e.au  
  inflating: tulips1/RawData/tulips1.A/raw/George12e.au  
  inflating: tulips1/RawData/tulips1.A/raw/George21e.au  
  inflating: tulips1/RawData/tulips1.A/raw/George22e.au  
  inflating: tulips1/RawData/tulips1.A/raw/George31e.au  
  inflating: tulips1/RawData/tulips1.A/raw/George32e.au  
  inflating: tulips1/RawData/tulips1.A/raw/George41e.au  
  inflating: tulips1/RawData/tulips1.A/raw/George42e.au  
  inflating: tulips1/RawData/tulips1.A/raw/Isaac11e.au  
  inflating: tulips1/RawData/tulips1.A/

  inflating: tulips1/RawData/tulips1.V/Candace31.00004  
  inflating: tulips1/RawData/tulips1.V/Candace31.00005  
  inflating: tulips1/RawData/tulips1.V/Candace31.00006  
  inflating: tulips1/RawData/tulips1.V/Candace31.00007  
  inflating: tulips1/RawData/tulips1.V/Candace31.00008  
  inflating: tulips1/RawData/tulips1.V/Candace31.00009  
  inflating: tulips1/RawData/tulips1.V/Candace32.00001  
  inflating: tulips1/RawData/tulips1.V/Candace32.00002  
  inflating: tulips1/RawData/tulips1.V/Candace32.00003  
  inflating: tulips1/RawData/tulips1.V/Candace32.00004  
  inflating: tulips1/RawData/tulips1.V/Candace32.00005  
  inflating: tulips1/RawData/tulips1.V/Candace32.00006  
  inflating: tulips1/RawData/tulips1.V/Candace32.00007  
  inflating: tulips1/RawData/tulips1.V/Candace32.00008  
  inflating: tulips1/RawData/tulips1.V/Candace32.00009  
  inflating: tulips1/RawData/tulips1.V/Candace41.00001  
  inflating: tulips1/RawData/tulips1.V/Candace41.00002  
  inflating: tulips1/RawData/tu

  inflating: tulips1/RawData/tulips1.V/George22.00003  
  inflating: tulips1/RawData/tulips1.V/George22.00004  
  inflating: tulips1/RawData/tulips1.V/George22.00005  
  inflating: tulips1/RawData/tulips1.V/George22.00006  
  inflating: tulips1/RawData/tulips1.V/George22.00007  
  inflating: tulips1/RawData/tulips1.V/George31.00001  
  inflating: tulips1/RawData/tulips1.V/George31.00002  
  inflating: tulips1/RawData/tulips1.V/George31.00003  
  inflating: tulips1/RawData/tulips1.V/George31.00004  
  inflating: tulips1/RawData/tulips1.V/George31.00005  
  inflating: tulips1/RawData/tulips1.V/George31.00006  
  inflating: tulips1/RawData/tulips1.V/George31.00007  
  inflating: tulips1/RawData/tulips1.V/George31.00008  
  inflating: tulips1/RawData/tulips1.V/George31.00009  
  inflating: tulips1/RawData/tulips1.V/George31.00010  
  inflating: tulips1/RawData/tulips1.V/George31.00011  
  inflating: tulips1/RawData/tulips1.V/George31.00012  
  inflating: tulips1/RawData/tulips1.V/George32.

  inflating: tulips1/RawData/tulips1.V/Jesse31.00002  
  inflating: tulips1/RawData/tulips1.V/Jesse31.00003  
  inflating: tulips1/RawData/tulips1.V/Jesse31.00004  
  inflating: tulips1/RawData/tulips1.V/Jesse31.00005  
  inflating: tulips1/RawData/tulips1.V/Jesse31.00006  
  inflating: tulips1/RawData/tulips1.V/Jesse31.00007  
  inflating: tulips1/RawData/tulips1.V/Jesse32.00001  
  inflating: tulips1/RawData/tulips1.V/Jesse32.00002  
  inflating: tulips1/RawData/tulips1.V/Jesse32.00003  
  inflating: tulips1/RawData/tulips1.V/Jesse32.00004  
  inflating: tulips1/RawData/tulips1.V/Jesse32.00005  
  inflating: tulips1/RawData/tulips1.V/Jesse32.00006  
  inflating: tulips1/RawData/tulips1.V/Jesse32.00007  
  inflating: tulips1/RawData/tulips1.V/Jesse32.00008  
  inflating: tulips1/RawData/tulips1.V/Jesse32.00009  
  inflating: tulips1/RawData/tulips1.V/Jesse41.00001  
  inflating: tulips1/RawData/tulips1.V/Jesse41.00002  
  inflating: tulips1/RawData/tulips1.V/Jesse41.00003  
  inflatin

  inflating: tulips1/RawData/tulips1.V/Simon42.00005  
  inflating: tulips1/RawData/tulips1.V/Simon42.00006  
  inflating: tulips1/RawData/tulips1.V/Simon42.00007  
  inflating: tulips1/RawData/tulips1.V/Simon42.00008  
  inflating: tulips1/RawData/tulips1.V/Simon42.00009  
  inflating: tulips1/README          


In [2]:
import numpy as np
from scipy.special import factorial
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
from load_data import get_data

In [148]:
from buffet import IndianBuffet
np.random.seed(100)

def init_ibm(data):
    Xs = len(data)
    return IndianBuffet(data, (1, 1, 1), [(1.7, 1, 1)] * Xs, [(0.5, 1, 1)] * Xs)

In [149]:
X_train, X_test = get_data(include_labels=True)

# Visual
print(len(X_train[0][0]))
# Audio
print(len(X_train[1][0]))
# Words
print(len(X_train[2][0]))

(64, 196)
(32, 196)
36
156
4


In [150]:
ibm = init_ibm([X_train[1], X_train[2]])

In [151]:
ibm.run_sampler(iters=5000)

10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
11


/Users/andzuck/Desktop/multimodal-indian-buffet/buffet.py:45: RuntimeWarning: invalid value encountered in log
  ll = self.D * np.log(sigma) + np.power(self.X[i] - mu, 2).sum() / sigma
/Users/andzuck/Desktop/multimodal-indian-buffet/buffet.py:167: RuntimeWarning: divide by zero encountered in log
  lpz = np.log(np.stack((self.N - mi, mi)))


11
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
1

11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
1

[array([[1., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0., 1., 0., 0.],
        [1., 1., 1., 0., 1., 0., 0., 1., 0., 1., 0.],
        [1., 0., 1., 1., 1., 1., 0., 1., 0., 1., 0.],
        [1., 0., 1., 0., 1., 1., 1., 0., 1., 0., 0.],
        [1., 0., 1., 1., 0., 0., 1., 0., 0., 1., 0.],
        [1., 1., 1., 0., 1., 0., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 1., 0., 0., 1., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0., 1., 1., 1.],
        [1., 1., 1., 0., 1., 0., 0., 1., 0., 1., 0.],
        [1., 0., 1., 1., 1., 1., 0., 1., 0., 1., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0.],
        [1., 1., 1., 0., 0., 1., 0., 0., 1., 0., 0.],
        [1., 1., 1., 0., 1., 0., 1., 1., 0., 0., 0.],
        [1., 0., 1., 1., 0., 1., 0., 0., 0., 1., 0.],
        [1., 0., 1., 0., 1., 1., 0., 0., 1., 0., 0.],
        [1., 1., 0., 0., 0.,

In [152]:
def predict(model, Xs):
    best = (-float('inf'), None, None)
    for Z in model.Zs:
        m = Z.sum(axis=0) - Z
        lps = np.sum(np.log(np.where(Z == 1, m, model.N - m) / model.N), axis=1)
        for f, X in zip(model.feats[:-1], Xs[:-1]):
            if X is not None:
                X_bar = Z @ f.weights(Z)
                diffs = np.sum((X - X_bar) ** 2, axis=1) / (2 * f.var_x)
                lps -= diffs + np.log(2 * np.pi) / 2 + np.log(f.var_x) * f.D / 2
        i = np.argmax(lps)
        if lps[i] > best[0]:
            best = lps[i], Z, i
    best_lp, best_Z, best_i = best
    W = model.feats[-1].weights(best_Z)
    X_w_pred = best_Z[best_i] @ W
    print(X_w_pred, Xs[-1])
#     print(np.argmax(X_w_pred) == np.argmax(Xs[-1]))
    return Xs[-1][np.argmax(X_w_pred)] == 1

In [153]:
data = [Z.shape for Z in ibm.Zs]
print(len(ibm.Zs))
print(data)

199
[(64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 11), (64, 

In [156]:
XS = X_train
N = len(XS[0])
print(N)

print(len(XS[0][0]))
print(len(XS[1][0]))
print(len(XS[2][0]))


# Vision
# print(sum(predict(ibm, [X[i] for X in [XS[0], XS[2]]]) for i in range(N)) / N)

# Audio
print(sum(predict(ibm, [X[i] for X in [XS[1], XS[2]]]) for i in range(N)) / N)

# # Multisensory in multisensory env
# print(sum(predict(ibm, [X[i] for X in [XS[0], XS[1], XS[2]]]) for i in range(N)) / N)
# # Multisensory in unisensory env (video)
# print(sum(predict(ibm, [X[i] for X in [XS[0], [None]*N, XS[2]]]) for i in range(N)) / N)
# # Multisensory in unisensory env (audio)
# print(sum(predict(ibm, [X[i] for X in [[None]*N, XS[1], XS[2]]]) for i in range(N)) / N)

64
36
156
4
[ 0.80173974 -0.09410949  0.22240667 -0.10804425] [1. 0. 0. 0.]
[0.2513448  0.33106848 0.1590756  0.12818726] [0. 1. 0. 0.]
[0.1372839  0.05456871 0.77167114 0.0465064 ] [0. 0. 1. 0.]
[ 0.13276023  0.05976669 -0.08129538  0.86613844] [0. 0. 0. 1.]
[ 0.76058212 -0.20005246 -0.05772538  0.51754583] [1. 0. 0. 0.]
[-0.00670016  0.87042224  0.10308043  0.10020789] [0. 1. 0. 0.]
[ 0.20099707  0.22297517  0.58397696 -0.06935642] [0. 0. 1. 0.]
[-0.07988854  0.08776496  0.12512798  0.88449339] [0. 0. 0. 1.]
[ 0.53503852  0.12881825  0.62692293 -0.25189886] [1. 0. 0. 0.]
[-0.03683484  0.59516088  0.3408717   0.07749348] [0. 1. 0. 0.]
[-0.06523415  0.1182702   0.85755193  0.10622478] [0. 0. 1. 0.]
[ 0.13276023  0.05976669 -0.08129538  0.86613844] [0. 0. 0. 1.]
[ 0.7860143  -0.22545179 -0.07736344  0.53386546] [1. 0. 0. 0.]
[-0.07107663  0.61914964  0.36059193  0.06601512] [0. 1. 0. 0.]
[ 0.17362938  0.54198088  0.27742555 -0.01876619] [0. 0. 1. 0.]
[-0.07308388  0.00696545  0.25174673